In [2]:
!pip install tensorflow-gpu
!pip install wandb

     |████████████████████████████████| 380.8MB 45kB/s 
     |████████████████████████████████| 450kB 42.6MB/s 
     |████████████████████████████████| 3.8MB 46.6MB/s 
     |████████████████████████████████| 81kB 12.7MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.2 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled goog

     |████████████████████████████████| 1.3MB 4.9MB/s 
     |████████████████████████████████| 102kB 13.7MB/s 
     |████████████████████████████████| 460kB 52.0MB/s 
     |████████████████████████████████| 92kB 12.2MB/s 
     |████████████████████████████████| 256kB 60.8MB/s 
     |████████████████████████████████| 92kB 13.7MB/s 
     |████████████████████████████████| 71kB 11.9MB/s 
     |████████████████████████████████| 184kB 52.8MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=914a08043433111bee8ace2ebc2876a553f41125850c8529ae4bb831611814bc
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for shortuuid: filename=shortuuid-0.5.0-cp36-none-any.whl size=5499 sha256=10a6173de48f1386f121e7b8b4591c71472c082c647d77358daf65172a2a72d6
  Stored in directory: /root/.cache/pip/wheels/3f/eb/fd/69e5177f67b505e44acbd1aedfbe44b91768ee0c4cd5636576
  Created wheel for watc

In [1]:
from google.colab import files
uploaded = files.upload()

MessageError: ignored

In [0]:
import pandas as pd
import numpy as np

import io

# df = pd.read_csv(io.BytesIO(uploaded['tweets.csv']))

# Get a pandas DataFrame object of all the data in the csv file:
url = 'https://raw.githubusercontent.com/lukas/ml-class/master/projects/6-rnn-timeseries/daily-min-temperatures.csv'
df = pd.read_csv(url, index_col=0)



In [11]:
!pip install plotutil

ERROR: Could not find a version that satisfies the requirement plotutil (from versions: none)
ERROR: No matching distribution found for plotutil


In [0]:
import matplotlib
matplotlib.use('Agg')  # noqa
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from tensorflow import keras
import numpy as np
import wandb


def fig2data(fig):
    """
    @brief Convert a Matplotlib figure to a 4D numpy array with RGBA channels and return it
    @param fig a matplotlib figure
    @return a numpy 3D array of RGBA values
    """
    # draw the renderer
    fig.canvas.draw()

    # Get the RGBA buffer from the figure
    w, h = fig.canvas.get_width_height()
    buf = np.fromstring(fig.canvas.tostring_argb(), dtype=np.uint8)
    buf.shape = (w, h, 4)

    # canvas.tostring_argb give pixmap in ARGB mode. Roll the ALPHA channel to have it in RGBA mode
    buf = np.roll(buf, 3, axis=2)
    return buf


def repeated_predictions(model, data, look_back, steps=100):
    predictions = []
    for i in range(steps):
        input_data = data[np.newaxis, :, np.newaxis]
        generated = model.predict(input_data)[0]
        data = np.append(data, generated)[-look_back:]
        predictions.append(generated)
    return predictions


class PlotCallback(keras.callbacks.Callback):
    def __init__(self, trainX, trainY, testX, testY, look_back, repeated_predictions=False):
        self.repeat_predictions = repeated_predictions
        self.trainX = trainX
        self.trainY = trainY
        self.testX = testX
        self.testY = testY
        self.look_back = look_back

    def on_epoch_end(self, epoch, logs):
        if self.repeat_predictions:
            preds = repeated_predictions(
                self.model, self.trainX[-1, :, 0], self.look_back, self.testX.shape[0])
        else:
            preds = self.model.predict(self.testX)

        # Generate a figure with matplotlib</font>
        figure = matplotlib.pyplot.figure(figsize=(10, 10))
        plot = figure.add_subplot(111)

        plot.plot(self.trainY)
        plot.plot(np.append(np.empty_like(self.trainY) * np.nan, self.testY))
        plot.plot(np.append(np.empty_like(self.trainY) * np.nan, preds))

        data = fig2data(figure)
        matplotlib.pyplot.close(figure)

        wandb.log({"image": wandb.Image(data)}, commit=False)

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import LSTM, SimpleRNN, Dropout
from tensorflow.keras.callbacks import LambdaCallback

import wandb
from wandb.keras import WandbCallback

# import plotutil
# from plotutil import PlotCallback

wandb.init()
config = wandb.config

In [0]:
# If repeated prediction is True, the green line in the wandb plot will correspond to
# using the past prediction as input to the next prediction (hard case).
# If repeated prediction is False, the green line in the wandb plot will correspond
# to make in a prediction off of ground truth data every time.
config.repeated_predictions = False
config.look_back = 20
data = df.Temp.astype('float32').values

# convert an array of values into a dataset matrix
def create_dataset(dataset):
    dataX, dataY = [], []
    for i in range(len(dataset)-config.look_back-1):
        a = dataset[i:(i+config.look_back)]
        dataX.append(a)
        dataY.append(dataset[i + config.look_back])
    return np.array(dataX), np.array(dataY)
    
# normalize data to between 0 and 1
# max_val = max(data)
# min_val = min(data)
# data=(data-min_val)/(max_val-min_val)

# split into train and test sets
split = int(len(data) * 0.70)
train = data[:split]
test = data[split:]

trainX, trainY = create_dataset(train)
testX, testY = create_dataset(test)

trainX = trainX[:, :, np.newaxis]
testX = testX[:, :, np.newaxis]


In [44]:
# create and fit the RNN
model = Sequential()
model.add(LSTM(10, input_shape=(config.look_back,1 )))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam', metrics=['mae'])
model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 10)                480       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 491
Trainable params: 491
Non-trainable params: 0
_________________________________________________________________


In [45]:
model.fit(trainX, trainY, epochs=100,  validation_data=(testX, testY),  callbacks=[WandbCallback(), PlotCallback(trainX, trainY, testX, testY, config.look_back, config.repeated_predictions)])


Train on 2534 samples, validate on 1074 samples
Epoch 1/100
2534/2534 [==============================] - 3s 1ms/sample - loss: 10.3477 - mae: 10.3477 - val_loss: 10.4104 - val_mae: 10.4104
Epoch 2/100
 768/2534 [========>.....................] - ETA: 0s - loss: 9.6626 - mae: 9.6626

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


2534/2534 [==============================] - 1s 276us/sample - loss: 8.7852 - mae: 8.7852 - val_loss: 8.3057 - val_mae: 8.3057
Epoch 3/100
2534/2534 [==============================] - 1s 273us/sample - loss: 6.7650 - mae: 6.7650 - val_loss: 6.5216 - val_mae: 6.5216
Epoch 4/100
2534/2534 [==============================] - 1s 293us/sample - loss: 5.5617 - mae: 5.5617 - val_loss: 5.5845 - val_mae: 5.5845
Epoch 5/100
2534/2534 [==============================] - 1s 281us/sample - loss: 4.8157 - mae: 4.8157 - val_loss: 4.8861 - val_mae: 4.8861
Epoch 6/100
2534/2534 [==============================] - 1s 292us/sample - loss: 4.2725 - mae: 4.2725 - val_loss: 4.3757 - val_mae: 4.3757
Epoch 7/100
2534/2534 [==============================] - 1s 279us/sample - loss: 3.8813 - mae: 3.8813 - val_loss: 3.9932 - val_mae: 3.9932
Epoch 8/100
2534/2534 [==============================] - 1s 289us/sample - loss: 3.5453 - mae: 3.5453 - val_loss: 3.6318 - val_mae: 3.6318
Epoch 9/100
2534/2534 [================